In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# settings
seed = 1
batch_size = 64
test_batch_size = 1000
momentum = 0.9
epochs = 50
log_interval = 100
lr = 0.01
save_model = False
num_classes = 10
criterion = nn.CrossEntropyLoss()
torch.backends.cudnn.benchmark=True

In [3]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.fc_size = 4 * 4 * 256    # for full connected size
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
#            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
#            nn.Dropout2d(0.2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
#            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
#            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(self.fc_size, 256),
            nn.ReLU(),
#            nn.ReLU(inplace=True),
            nn.Dropout2d(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
#            nn.ReLU(inplace=True),
#            nn.Dropout2d(0.2),
            nn.Linear(64, num_classes),
        )
        

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [4]:
def train(model, device, train_loader, optimizer, log_interval, epoch, epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        # proccessing display for training
        if batch_idx % log_interval == 0:
            print('Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, epochs, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).sum().item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    # proccessing display for test   
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
# use device to Cuda or CPU
use_cuda =  torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

torch.manual_seed(seed)


# get datasets
train_dataset=datasets.CIFAR10('./data_c', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
                       
                   ]))

test_dataset=datasets.CIFAR10('./data_c', train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
    ]))


# check datasets shape
image = train_dataset[0]
labels = [i[-1] for i in train_dataset]
print(image[0].size())
print('categolize Labels:', max(labels) + 1)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([3, 32, 32])
categolize Labels: 10


In [7]:
# datasets to DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=test_batch_size, shuffle=False, num_workers=2)

# check tensor shape
for images, labels in train_loader:
    print(images.size())
    print(images[0].size())
    print(labels.size())
    
    break

torch.Size([64, 3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([64])


In [8]:
# build model
model = CNNNet().to(device)
optimizer = optim.SGD(model.parameters(),
                       lr=lr,
                       momentum=momentum,
                       weight_decay=5e-4,
                      )

# model run
def run():
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, log_interval, epoch, epochs)
        test(model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cnn.pt")

In [9]:
%%time
print('Use device is', str(device).upper())

run()

Use device is CUDA
Train Epoch: 1/50 [0/50000 (0%)]	Loss: 2.310911
Train Epoch: 1/50 [6400/50000 (13%)]	Loss: 2.298832
Train Epoch: 1/50 [12800/50000 (26%)]	Loss: 2.312756
Train Epoch: 1/50 [19200/50000 (38%)]	Loss: 2.168451
Train Epoch: 1/50 [25600/50000 (51%)]	Loss: 2.096751
Train Epoch: 1/50 [32000/50000 (64%)]	Loss: 1.946207
Train Epoch: 1/50 [38400/50000 (77%)]	Loss: 2.038442
Train Epoch: 1/50 [44800/50000 (90%)]	Loss: 1.728067

Test set: Average loss: 0.0017, Accuracy: 3808/10000 (38%)

Train Epoch: 2/50 [0/50000 (0%)]	Loss: 1.557467
Train Epoch: 2/50 [6400/50000 (13%)]	Loss: 1.637969
Train Epoch: 2/50 [12800/50000 (26%)]	Loss: 1.596716
Train Epoch: 2/50 [19200/50000 (38%)]	Loss: 1.604777
Train Epoch: 2/50 [25600/50000 (51%)]	Loss: 1.702741
Train Epoch: 2/50 [32000/50000 (64%)]	Loss: 1.670536
Train Epoch: 2/50 [38400/50000 (77%)]	Loss: 1.603898
Train Epoch: 2/50 [44800/50000 (90%)]	Loss: 1.249111

Test set: Average loss: 0.0014, Accuracy: 4982/10000 (50%)

Train Epoch: 3/50 [0/50

Train Epoch: 18/50 [0/50000 (0%)]	Loss: 0.261118
Train Epoch: 18/50 [6400/50000 (13%)]	Loss: 0.277738
Train Epoch: 18/50 [12800/50000 (26%)]	Loss: 0.142400
Train Epoch: 18/50 [19200/50000 (38%)]	Loss: 0.116306
Train Epoch: 18/50 [25600/50000 (51%)]	Loss: 0.306658
Train Epoch: 18/50 [32000/50000 (64%)]	Loss: 0.325059
Train Epoch: 18/50 [38400/50000 (77%)]	Loss: 0.225291
Train Epoch: 18/50 [44800/50000 (90%)]	Loss: 0.285768

Test set: Average loss: 0.0007, Accuracy: 7804/10000 (78%)

Train Epoch: 19/50 [0/50000 (0%)]	Loss: 0.207646
Train Epoch: 19/50 [6400/50000 (13%)]	Loss: 0.097505
Train Epoch: 19/50 [12800/50000 (26%)]	Loss: 0.238980
Train Epoch: 19/50 [19200/50000 (38%)]	Loss: 0.223505
Train Epoch: 19/50 [25600/50000 (51%)]	Loss: 0.221342
Train Epoch: 19/50 [32000/50000 (64%)]	Loss: 0.294252
Train Epoch: 19/50 [38400/50000 (77%)]	Loss: 0.224149
Train Epoch: 19/50 [44800/50000 (90%)]	Loss: 0.285597

Test set: Average loss: 0.0008, Accuracy: 7762/10000 (78%)

Train Epoch: 20/50 [0/5000

KeyboardInterrupt: 